# YouTube - summary generator
Use YT videos transcript and summarize it

In [7]:
! pip install youtube-transcript-api
! pip install pytube

  Obtaining dependency information for youtube-transcript-api from https://files.pythonhosted.org/packages/33/c1/18e32c7cd693802056f385c3ee78825102566be94a811b6556f17783c743/youtube_transcript_api-0.6.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.8 MB/s eta 0:00:00


In [3]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
load_dotenv() # Load environment variables from .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # Get API key from environment variable
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__7fe96d9dfa664e99ad0e78c2f9302178"
os.environ["LANGCHAIN_PROJECT"] = "youtube_template"

0. YouTube Search package

In [1]:
#! pip install youtube_search

In [3]:
from langchain.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

In [5]:
tool.run("lex friedman,5")

"['/watch?v=IQBA4aytp_U&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=EG7I6Bt_NZY&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=eTBAxD6lt2g&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=gPfriiHBBek&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=Mde2q7GFCrw&pp=ygUMbGV4IGZyaWVkbWFu']"

1. Simple Video

In [38]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=pNcQ5XXMgH4", add_video_info=True)

In [39]:
result = loader.load()

In [40]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print(result[0].metadata)
print ("")
print (result)

<class 'list'>
Found video from Greg Kamradt (Data Indy) that is 668 seconds long
{'source': 'pNcQ5XXMgH4', 'title': 'LangChain 101: YouTube Transcripts + OpenAI', 'description': 'Unknown', 'view_count': 17764, 'thumbnail_url': 'https://i.ytimg.com/vi/pNcQ5XXMgH4/hqdefault.jpg?sqp=-oaymwEXCJADEOABSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLCmP9TXvB4nm22ZX7b5Tl0AagEU3A', 'publish_date': '2023-02-23 00:00:00', 'length': 668, 'author': 'Greg Kamradt (Data Indy)'}

[Document(page_content="what is going on good people again right now we have a super exciting tutorial because we are going to take YouTube transcripts and we're going to pass them to open Ai and the way that we're going to do that is via a library called Lang chain which is what this entire series is about now before we jumped into it I wanted to show a diagram again I think these diagrams are helpful but you have to let me know so just let me know in the comments here so I wanted to do an overview about what we're actually going to be w

In [6]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [43]:
# Summarize
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
video_summary = chain.run(result)

In [44]:
video_summary

' This tutorial explains how to use the Lang Chain library to take YouTube transcripts and pass them to Open AI to generate a summary. It also explains how to use the recursive character splitter to split up long transcripts into smaller chunks, and how to use the mapreduce method to generate a summary of multiple videos. Finally, it explains how to use the summarize scan to generate a summary of multiple videos.'

### 2. Long Video (use map-reduce)

    When video is too long, it will not fit into context window. Use map-reduce chain using chanks of transcript

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(result)

In [20]:
len(texts)

14

In [21]:
#print(texts)
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"what is going on good people again right now we have a super exciting tutorial because we are going to take YouTube transcripts and we're going to pass them to open Ai and the way that we're going to do that is via a library called Lang chain which is what this entire series is about now before we jumped into it I wanted to show a diagram again I think these diagrams are helpful but you have to let me know so just let me know in the comments here so I wanted to do an overview about what we're actually going to be writing out in code because I think it's a little easier to see in pictures first so the way this is going to work is we're going to have a video a YouTube video we're going to pass it we're going to pass it a URL and then what Lang chain is going to help us do is it's going to help us load this video as a document and a doc

' This tutorial explains how to use Open AI and Lang Chain to generate summaries of multiple documents, such as YouTube videos. It provides a diagram to help visualize the code and explains how to use the mapreduce method to combine documents into one concise summary. The speaker also encourages viewers to leave comments about how the videos can be improved and about their own business problems.'

### 3. Multiple Videos

In [4]:
youtube_url_list = ["https://www.youtube.com/watch?v=UfL7hqGBLAQ", "https://www.youtube.com/watch?v=9z7p28FhoEc"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    print(result[0].metadata)
    
    texts.extend(text_splitter.split_documents(result))

{'source': 'UfL7hqGBLAQ', 'title': 'This is the biggest political corruption scandal of our lifetime: Elise Stefanik', 'description': 'Unknown', 'view_count': 65586, 'thumbnail_url': 'https://i.ytimg.com/vi/UfL7hqGBLAQ/hq720.jpg', 'publish_date': '2023-09-13 00:00:00', 'length': 344, 'author': 'Fox News'}
{'source': '9z7p28FhoEc', 'title': 'Morocco earthquake: Search for survivors continues - BBC News', 'description': 'Unknown', 'view_count': 74481, 'thumbnail_url': 'https://i.ytimg.com/vi/9z7p28FhoEc/hq720.jpg', 'publish_date': '2023-09-13 00:00:00', 'length': 393, 'author': 'BBC News'}


In [7]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

" Congressman Elise Stefanik of New York's 21st Congressional District is supportive of the impeachment inquiry into President Biden and is committed to helping re-elect President Trump in 2024. A search and rescue operation is underway in Morocco following a devastating earthquake, with UK Aid workers searching for survivors. The Moroccan government has taken 48 hours to invite aid from four countries, and there has been criticism that other international agencies and countries have not been able to get into Morocco. Volunteers, family, friends, local charities, and NOS are helping people in the mountains."

### 4. Extract topic title & Short Description

In [8]:
# LangChain basics
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import create_extraction_chain

# Vector Store and retrievals
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
#from langchain.vectorstores import Pinecone
#import pinecone

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()

True

Let's start with our map prompt which will iterate over the chunks we just made

In [9]:
template="""
You are a helpful assistant that helps retrieve topics talked about in a transcript
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics include:
  - Themes
  - Business Ideas
  - Interesting Stories
  - Money making businesses
  - Quick stories about people
  - Mental Frameworks
  - Stories about an industry
  - Analogies mentioned
  - Advice or words of caution
  - Pieces of news or current events
- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the transcript
- Do not respond with anything outside of the transcript. If you don't see any topics, say, 'No Topics'
- Do not respond with numbers, just bullet points
- Do not include anything about 'Marketing Against the Grain'
- Only pull topics from the transcript. Do not use the examples
- Make your titles descriptive but concise. Example: 'Shaan's Experience at Twitch' should be 'Shaan's Interesting Projects At Twitch'
- A topic should be substantial, more than just a one-off comment

% START OF EXAMPLES
 - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
 - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
 - Revenge Against The Spam Calls: A couple of businesses focused on protecting consumers: RoboCall, TrueCaller, DoNotPay, FitIt
 - Wildcard CEOs vs. Prudent CEOs: However, Munger likes to surround himself with prudent CEO's and says he would never hire Musk.
 - Chess Business: Priyav, a college student, expressed his doubts on the MFM Facebook group about his Chess training business, mychesstutor.com, making $12.5K MRR with 90 enrolled.
 - Restaurant Refiller: An MFM Facebook group member commented on how they pay AirMark $1,000/month for toilet paper and toilet cover refills for their restaurant. Shaan sees an opportunity here for anyone wanting to compete against AirMark.
 - Collecting: Shaan shared an idea to build a mobile only marketplace for a collectors' category; similar to what StockX does for premium sneakers.
% END OF EXAMPLES
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

Then we have our combine prompt which will run once over the results of the map prompt above

In [10]:
template="""
You are a helpful assistant that helps retrieve topics talked about in transcript
- You will be given a series of bullet topics of topics vound
- Your goal is to exract the topic names and brief 1-sentence description of the topic
- Deduplicate any bullet points you see
- Only pull topics from the transcript. Do not use the examples

% START OF EXAMPLES
 - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
 - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
% END OF EXAMPLES
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

Create your LLMs and get data

In [11]:
# Creating two versions of the model so I can swap between gpt3.5 and gpt4
llm3 = ChatOpenAI(temperature=0,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )

llm4 = ChatOpenAI(temperature=0,
                  model_name="gpt-4-0613",
                  request_timeout = 180
                 )

In [12]:
# Use YT transcript
print (f"You have {len(texts)} docs. First doc is {llm3.get_num_tokens(texts[0].page_content)} tokens")

You have 6 docs. First doc is 582 tokens


Store YouTube transcripts in Chroma to later use for retrival

In [54]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.base import Embeddings
import chromadb
from chromadb.utils import embedding_functions
import uuid

client = chromadb.HttpClient(host="20.115.73.2", port=8000)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    model_name="text-embedding-ada-002"
)

collection = client.get_or_create_collection(
            name="you_tube", 
            embedding_function=openai_ef
        )
for text in texts:
    collection.add(
        ids=[str(uuid.uuid1())], 
        metadatas=text.metadata, 
        documents=text.page_content
    )

#### Step 1: Extract topic title and Description

In [13]:
chain = load_summarize_chain(llm4,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine,
                             verbose=True
                            )

In [14]:
topics_found = chain.run({"input_documents": texts})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant that helps retrieve topics talked about in a transcript
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics include:
  - Themes
  - Business Ideas
  - Interesting Stories
  - Money making businesses
  - Quick stories about people
  - Mental Frameworks
  - Stories about an industry
  - Analogies mentioned
  - Advice or words of caution
  - Pieces of news or current events
- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the transcript
- Do not respond with anything outside of the transcript. If you don't see any topics, say, 'No Topics'
- Do not respond with numbers, just bullet points
- Do not include anything about 'Marketing Against the Grain'
- Only pull topics from the transcript. Do not u

In [15]:
print (topics_found)

- Impeachment Inquiry into President Biden: An impeachment inquiry has been initiated against President Biden due to alleged involvement in his son's business dealings.
- Alleged Bribery Scheme with Burisma and Obama Administration: There are allegations of a bribery scheme involving Burisma and the Obama administration's Ukraine policy.
- Importance of Impeachment Inquiry: The impeachment inquiry is crucial for maintaining Congress's power over deposition and subpoena.
- Bribery Scheme and Corruption Allegations: The speaker discusses allegations of bribery and corruption at the highest level of government.
- Underestimation by Media: The speaker suggests that the media often underestimates the gravity of certain situations.
- Donald Trump's Leadership and Potential Running Mate: The speaker expresses admiration for Donald Trump and hints at a potential partnership.
- New York's Immigration Issue: The speaker discusses a controversial issue in New York regarding work rights for illega

Structured Data - Turn your LLM output into structured data

In [16]:
schema = {
    "properties": {
        # The title of the topic
        "topic_name": {
            "type": "string",
            "description" : "The title of the topic listed"
        },
        # The description
        "description": {
            "type": "string",
            "description" : "The description of the topic listed"
        },
        "tag": {
            "type": "string",
            "description" : "The type of content being described",
            "enum" : ['Business Models', 'Life Advice', 'Health & Wellness', 'Stories', 'Politics']
        }
    },
    "required": ["topic", "description"],
}

In [17]:
# Using gpt3.5 here because this is an easy extraction task and no need to jump to gpt4
chain = create_extraction_chain(schema, llm3)

In [18]:
topics_structured = chain.run(topics_found)

In [19]:
topics_structured

[{'topic_name': 'Impeachment Inquiry into President Biden',
  'description': "An impeachment inquiry has been initiated against President Biden due to alleged involvement in his son's business dealings.",
  'tag': 'Politics'},
 {'topic_name': 'Alleged Bribery Scheme with Burisma and Obama Administration',
  'description': "There are allegations of a bribery scheme involving Burisma and the Obama administration's Ukraine policy.",
  'tag': 'Politics'},
 {'topic_name': 'Importance of Impeachment Inquiry',
  'description': "The impeachment inquiry is crucial for maintaining Congress's power over deposition and subpoena.",
  'tag': 'Politics'},
 {'topic_name': 'Bribery Scheme and Corruption Allegations',
  'description': 'The speaker discusses allegations of bribery and corruption at the highest level of government.',
  'tag': 'Politics'},
 {'topic_name': 'Underestimation by Media',
  'description': 'The speaker suggests that the media often underestimates the gravity of certain situations

### Expand on each topics

In [20]:
# The system instructions. Notice the 'context' placeholder down below. This is where our relevant docs will go.
# The 'question' in the human message below won't be a question per se, but rather a topic we want to get relevant information on
system_template = """
You will be given transcript which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

# This will pull the two messages together and get them ready to be sent to the LLM through the retriever
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [21]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import chromadb
from chromadb.utils import embedding_functions


client = chromadb.HttpClient(host="20.115.73.2", port=8000)
        
index = Chroma(
    client=client,
    collection_name="you_tube",
    embedding_function=OpenAIEmbeddings()
)

In [22]:
# I'm using gpt4 for the increased reasoning power.
# I'm also setting k=4 so the number of relevant docs we get back is 4. This parameter should be tuned to your use case
qa = RetrievalQA.from_chain_type(llm=llm4,
                                 chain_type="stuff",
                                 retriever=index.as_retriever(k=4),
                                 chain_type_kwargs = {
#                                      'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 })

Then let's iterate through the topics that we found and run our QA query on them.

This will print out our expanded topics. This is the final result you can use wherever you want!

In [23]:
# Only doing the first 3 for conciseness 
for topic in topics_structured[:3]:
    query = f"""
        {topic['topic_name']}: {topic['description']}
    """
 
    expanded_topic = qa.run(query)

    print(f"{topic['topic_name']}: {topic['description']}")
    print(expanded_topic)
    print ("\n\n")

Impeachment Inquiry into President Biden: An impeachment inquiry has been initiated against President Biden due to alleged involvement in his son's business dealings.
The impeachment inquiry into President Biden is centered around his alleged involvement in his son, Hunter's business dealings. The inquiry, supported by Republican Conference Chair Elise Stefanik, is investigating over 20 LLCs used by the Biden family and over $20 million from various countries going into the Biden family. The inquiry also involves a supposed bribery scheme related to Burisma and the Obama administration's policy for Ukraine. Stefanik believes this could be the largest political corruption scandal in the country's history. The impeachment inquiry is seen as a crucial step to ensure transparency and uncover the truth.



Alleged Bribery Scheme with Burisma and Obama Administration: There are allegations of a bribery scheme involving Burisma and the Obama administration's Ukraine policy.
An impeachment inq

Based on the expanded topics, write a custom script

In [24]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)

prompt = PromptTemplate(
    input_variables=["topics"],
    template="""
        I want you to act as a stand-up comedian. I will provide you with some topics related to current events 
        and you will use your wit, creativity, and observational skills to create a routine based on those topics. 
        You should also be sure to incorporate personal anecdotes or experiences into the routine in order to 
        make it more relatable and engaging for the audience. My first request is "I want an humorous take on politics."
        TOPICS: {topics}
        """,
)

In [25]:
# Using simple chain
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
comedy_script = chain.run(expanded_topic)

In [26]:
print(comedy_script)


        Alright everyone, let's talk politics! So, you know the whole impeachment inquiry going on against President Biden right now? Well, apparently now the House Republicans have discovered a whopping TWENTY LLCs used by the Biden family, if you can believe it. Ya know, with Biden being so famous for his "middle class Joe" shtick, you'd think he wouldn't need *that* many LLCs! But seriously, this is really disconcerting. When there's twenty million dollars from foreign countries showing up in the Biden accounts, that's when things start to get a little sketchy. 

But what I'm most curious about is what the House Republicans are going to do with this information. I mean, impeachment? Come on, that's a little extreme! I mean, I guess it's better than turning a blind eye and nothing being done about it. But I'd like to think that there's more transparency in our government than that.

But you know what they say; even poor people still want justice - haha! Alright, well, that's all I'v

In [27]:
# Try GPT4
chain = LLMChain(llm=llm4, prompt=prompt)

llm4_comedy_script = chain.run(expanded_topic)

In [28]:
print(llm4_comedy_script)

Alright, alright, settle down folks. So, let's talk about politics, shall we? Now, I don't know about you, but I find politics to be a lot like my ex-wife. It's confusing, it's messy, and just when you think you've got it figured out, it takes half your stuff and leaves you questioning your sanity.

Now, we've got this impeachment inquiry into President Biden, right? Apparently, he's been accused of getting involved in his son Hunter's business dealings. I mean, come on, who among us hasn't meddled in our kids' lives a little too much? I once tried to help my son with his science project and ended up setting the kitchen on fire. But hey, at least I didn't get impeached for it!

And then there's this whole thing about a potential bribery scheme related to Burisma and the Obama administration's policy for Ukraine. I mean, I can barely keep track of my own finances, let alone understand international bribery schemes. I once tried to bribe my way out of a parking ticket with a half-eaten s

In [45]:
# Save script
f = open("llm4_comedy_script.txt", "w")
f.write(llm4_comedy_script)
f.close()

In [46]:
# Translate to Polish
prompt_2_pl = PromptTemplate(
    input_variables=["script"],
    template="""
        Translate transcript below to polish:
        TRANSCRIPT: {script}
        """,
)

In [48]:
# Using simple chain
from langchain.chains import LLMChain
chain = LLMChain(llm=llm4, prompt=prompt_2_pl)

# Run the chain only specifying the input variable.
comedy_script_in_pl = chain.run(llm4_comedy_script)

In [50]:
print(comedy_script_in_pl)

TRANSCRIPT: Dobrze, dobrze, uspokójmy się wszyscy. Więc, porozmawiajmy o polityce, co? Nie wiem jak wy, ale dla mnie polityka jest jak moja była żona. Jest skomplikowana, jest bałaganem, i kiedy myślisz, że już ją zrozumiałeś, zabiera ci połowę twoich rzeczy i pozostawia cię z pytaniem o twoje zdrowie psychiczne.

Teraz mamy to śledztwo w sprawie impeachmentu prezydenta Bidena, prawda? Podobno został oskarżony o ingerencję w interesy swojego syna Huntera. No dajcie spokój, kto z nas nie mieszał się za bardzo w życie swoich dzieci? Kiedyś próbowałem pomóc mojemu synowi w projekcie z nauk przyrodniczych i skończyło się na tym, że podpaliłem kuchnię. Ale przynajmniej nie zostałem za to postawiony w stan oskarżenia!

A potem jest cała ta sprawa o potencjalnym przekręcie związanym z Burismą i polityką administracji Obamy dla Ukrainy. Przecież ledwo daję radę śledzić swoje własne finanse, nie mówiąc już o zrozumieniu międzynarodowych schematów łapówkarskich. Kiedyś próbowałem przekupić straż

### Create Tweeter thread based on topics:

In [51]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)

prompt = PromptTemplate(
    input_variables=["topics"],
    template="""
        Craft a Tweeter thread based on the context below. 
        I want you to act as a social media influencer. I will provide you with some topics related to current events 
        and you will use your wit, creativity, and observational skills to create a a Tweeter thread based on those topics. 
        You should also be sure to incorporate personal anecdotes or experiences into the routine in order to 
        make it more relatable and engaging for the audience."
        TOPICS: {topics}
        """,
)

In [53]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
tweeter_thread = chain.run(expanded_topic)

In [54]:
print(tweeter_thread)


Thread:
1. Have you heard about the impeachment inquiry into President Biden? It's related to some serious allegations, including possible involvement of the Biden family in business dealings and a potential bribery scheme involving a Ukranian company. 
2. It's being reported that House Republicans have uncovered over 20 LLCs used by the Biden family and that there were over $20 million dollars coming from various countries and going into the Biden family. 
3. While these are serious allegations, the impeachment inquiry is seen as an important step to ensure Congress does its job and has the power to deposition and subpoena, and to bring transparency to the American people about potential corruption at the highest level of government. 
4. I know so much about this topic, not just from following the news, but also because I've personally seen the same kind of behavior on a much smaller scale. It's so sad to see the potential for this type of corruption at the highest level. 
5. We dese

### Text-to-Speach

text 2 speach:
https://thepythoncode.com/article/convert-text-to-speech-in-python

In [29]:
import gtts
from playsound import playsound

In [34]:
tts = gtts.gTTS(llm4_comedy_script)

In [35]:
# save the audio file
tts.save("llm4_comedy_script.mp3")